In [ ]:
# Import required libraries and dependencies
import pandas as pd
import numpy as np
import hvplot.pandas
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
scaler = StandardScaler().fit(df_market_data)
market_data_scaled = scaler.transform(df_market_data)
#scaled_data = scaler.fit_transform(df_market_data)
#scaled_data = scaler.fit_transform(df_market_data)





In [ ]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(
    market_data_scaled,
    columns=df_market_data.columns,
    index=df_market_data.index
)

# Display the first 5 rows of the new DataFrame
df_market_data_scaled.head(10)

# Copy the crypto names from the original data
crypto_names = df_market_data.index.copy()

# Set the coinid column as index
df_market_data_scaled.index.name = "coin_id"

# Display sample data
df_market_data_scaled.head()


---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k_values = [k for k in range(1, 12)]


In [ ]:
# Create an empty list to store the inertia values
inertia_values = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=0)
    kmeans.fit(df_market_data_scaled)
    inertia_values.append(kmeans.inertia_)


In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {k_values[i]: inertia_values[i] for i in range(len(k_values))}


# Create a DataFrame with the data to plot the Elbow curve
df_elbow_data = pd.DataFrame(elbow_data.items(), columns=["k", "inertia"])



In [ ]:
# Plot a line chart with all the inertia values computed with the different values of k to visually identify the optimal value for k.
plt.plot(k_values, inertia_values, marker='o')

# Set the plot title and axis labels
plt.title('Elbow curve')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')

# Show the plot
plt.show()


#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer: Based on the Elbow curve, it appears that the optimal value for k is 3, as this is where the rate of decrease in inertia starts to slow down significantly.

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
k = 3
kmeans = KMeans(n_clusters=k, random_state=0)


In [ ]:
# Fit the K-Means model using the scaled data
kmeans.fit(df_market_data)


In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
cluster_labels = kmeans.predict(df_market_data)
#Add the cluster labels to the original dataset.
df_market_data['cluster'] = cluster_labels

#Analyze the characteristics of each cluster.
df_market_data.groupby('cluster').mean()

# Print the resulting array of cluster values.
print(kmeans.labels_)



In [ ]:
# Create a copy of the DataFrame
df_market_data_clusters = df_market_data.copy()


In [ ]:
# Add a new column to the DataFrame with the predicted clusters
df_market_data['cluster'] = kmeans.labels_

# Display sample data
print(df_market_data.head())


In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 

scatter_plot = df_market_data.hvplot.scatter(
x="price_change_percentage_24h",
y="price_change_percentage_7d",
by="cluster",
hover_cols=["coin_id"]
)




In [ ]:
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
df_market_data_clusters.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="cluster",
    hover_cols=["coin_name"],
    width=800,
    height=400
)


---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.

pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.


from sklearn.decomposition import PCA

# Create a PCA model instance with n_components=3
pca = PCA(n_components=3)

# Use the fit_transform method on the scaled data
market_data_pca = pca.fit_transform(df_market_data_scaled)

# Create a DataFrame with the PCA data and the cluster labels
df_market_data_pca = pd.DataFrame(
    data=market_data_pca,
    columns=["principal component 1", "principal component 2", "principal component 3"],
    index=df_market_data.index
)
df_market_data_pca["cluster"] = cluster_labels

# View the first five rows of the DataFrame. 
df_market_data_pca.head()


In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
# Create a PCA model instance and set n_components=3
pca = PCA(n_components=3)

# Fit the PCA model to the scaled data and transform it
pca_data = pca.fit_transform(df_market_data_scaled)

# Retrieve the explained variance of the principal components
explained_variance = pca.explained_variance_ratio_

# Print the explained variance of each principal component
print(explained_variance)


#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** The total explained variance of the three principal components is 0.9417, which means that these three principal components contain about 94.17% of the total information in the data.

In [ ]:
# Create a PCA model instance and set `n_components=3`
pca = PCA(n_components=3)

# Use the PCA model with `fit_transform` to reduce to three principal components
pca_transformed_data = pca.fit_transform(df_market_data_scaled)

# Creating a DataFrame with the PCA data
df_market_data_pca = pd.DataFrame(
    pca_transformed_data,
    columns=["principal component 1", "principal component 2", "principal component 3"],
    index=df_market_data.index
)

# Copy the crypto names from the original data
df_market_data_pca["CoinId"] = df_market_data.index

# Set the coinid column as index
df_market_data_pca.set_index("CoinId", inplace=True)

# Display sample data
print(df_market_data_pca.head())


In [ ]:
# Create a new DataFrame with the PCA data.
df_market_data_pca = pd.DataFrame(
    pca_transformed_data,
    columns=["principal component 1", "principal component 2", "principal component 3"],
    index=df_market_data.index
)

# Copy the crypto names from the original data
crypto_names = df_market_data.index.copy()

# Set the coinid column as index
df_market_data_pca.index.name = "coin_id"

# Display sample data
df_market_data_pca.head()



---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11

k_values = [k for k in range(1, 12)]

In [ ]:
# Create an empy list to store the inertia values
inertia_values_pca = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list

for k in k_values:
    kmeans_pca = KMeans(n_clusters=k, random_state=0)
    kmeans_pca.fit(df_market_data_pca)
    inertia_values_pca.append(kmeans_pca.inertia_)


In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data = {k_values[i]: inertia_values[i] for i in range(len(k_values))}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow_data = pd.DataFrame(elbow_data.items(), columns=["k", "inertia"])  


In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
plt.plot(k_values, inertia_values, marker='o')

# Set the plot title and axis labels
plt.title('Elbow curve')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Inertia')

# Show the plot
plt.show()


#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:**Based on the Elbow curve, it appears that the optimal value for k is 3.

* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** Based on my visual inspection, the k value in both plots seems to me the same. 

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Fit the K-Means model using the PCA data
kmeans_pca.fit(df_market_data_pca)

In [ ]:

# Predict the clusters to group the cryptocurrencies using the PCA data
crypto_pca_clusters = kmeans_pca.predict(df_market_data_pca)

# Print the resulting array of cluster values
print(crypto_pca_clusters)



In [ ]:
# Create a copy of the DataFrame with the PCA data
df_pca_clusters = df_market_data_pca.copy()

# Add a new column to the DataFrame with the predicted clusters
df_pca_clusters['crypto_cluster'] = kmeans_pca.labels_

# Display sample data
df_pca_clusters.head()



In [ ]:
# Creating a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    clustered_df,
    x="principal component 1",
    y="principal component 2",
    z="principal component 3",
    hover_data=["Algorithm"],
    hover_name="CoinName",
    color="Class",
    symbol="Class",
    width=800
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means by setting by="crypto_cluster"
# add the crypto name in the `hover_cols` parameter to identify the cryptocurrency represented by each data point.
# the marker parameter should be set to ["hex", "square", "cross", "inverted_triangle"]
# Lastly, set the title


#df_market_data_pca.hvplot.scatter(
   # x="principal component 1",
  #  y="principal component 2",
  #  by="crypto_cluster",
  #  marker=["hex", "square", "cross", "inverted_triangle"],
  #  hover_cols=["coinid"],
  #  title="Crypto Clusters"
#)#

df_market_data_pca.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="crypto_cluster",
    marker=["hex", "square", "cross", "inverted_triangle"],
    hover_cols=["coinid"],
    title="Crypto Clusters"
)


In [ ]:
df_pca_clusters = df_pca_clusters.rename(columns={'cluster': 'crypto_cluster'})
df_pca_clusters

In [ ]:
# Perform K-Means clustering on the PCA data
kmeans = KMeans(n_clusters=4, random_state=0)
kmeans.fit(X_pca)

# Predict the clusters
labels = kmeans.fit_predict(X_pca)

# Create a copy of the DataFrame with the PCA data
pca_df = pd.DataFrame(X_pca, columns=["PC1", "PC2"], index=crypto_df.index)

# Add a new column to the DataFrame with the predicted clusters
clustered_df = pca_df.copy()
clustered_df["crypto_cluster"] = labels

# Create a scatter plot using hvPlot
clustered_df.hvplot.scatter(
    x="PC1",
    y="PC2",
    by="crypto_cluster",
    hover_cols=["coinid"],
    marker=["hex", "square", "cross", "inverted_triangle"],
    title="Crypto Clusters"
)


In [ ]:
# Apply PCA to reduce dimensions to 2
pca = PCA(n_components=2, random_state=0)
X_pca = pca.fit_transform(X)

# Perform K-Means clustering on the PCA data
kmeans = KMeans(n_clusters=4, random_state=0)
kmeans.fit(X_pca)

# Predict the clusters
labels = kmeans.predict(X_pca)

# Add the predicted clusters to the original dataframe
df["crypto_cluster"] = labels


In [ ]:
# Initialize the K-Means model using the best value for k
#kmeans_pca = KMeans(n_clusters=3, random_state=0)
kmeans = KMeans(n_clusters=3, random_state=0)
kmeans.fit(pca1)

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves

In [ ]:
# Composite plot to contrast the clusters
# YOUR CODE HERE!

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** 